## The `taskyield` Construct

The following example illustrates the use of the `taskyield`  directive.  The tasks in the example compute something useful and then do some computation  that must be done in a critical region. By using `taskyield` when a task  cannot get access to the `critical` region the implementation can suspend  the current task and schedule some other task that can do something useful.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: taskyield.1c
* type: C
* version: omp_3.1
*/
#include <omp.h>

void something_useful ( void );
void something_critical ( void );
void foo ( omp_lock_t * lock, int n )
{
   int i;

   for ( i = 0; i < n; i++ )
      #pragma omp task
      {
          something_useful();
          while ( !omp_test_lock(lock) ) {
             #pragma omp taskyield
          }
          something_critical();
          omp_unset_lock(lock);
      }
}



In [ ]:

! name: taskyield.1f
! type: F-free
! version:    omp_3.1
subroutine foo ( lock, n )
   use omp_lib
   integer (kind=omp_lock_kind) :: lock
   integer n
   integer i

   do i = 1, n
     !$omp task
       call something_useful()
       do while ( .not. omp_test_lock(lock) )
         !$omp taskyield
       end do
       call something_critical()
       call omp_unset_lock(lock)
     !$omp end task
   end do

end subroutine

